In [40]:
import classla
import pandas as pd
import numpy as np
import re

#classla.download('hr')
#classla.download('hr', type='nonstandard')

In [41]:
def is_float(element):
    try:
        float(element)
        return True
    except ValueError:
        return False

In [42]:
artist_name = "Plavi Orkestar"
df = pd.read_csv(f'data/{artist_name}_final.csv')



In [43]:
df.Lyrics_lyricstranslate.isnull().sum()

9

In [44]:
df.Lyrics_lyricstranslate.fillna(df.Lyrics_cuspajz).isnull().sum()


7

In [45]:
df.Lyrics_lyricstranslate.fillna(df.Lyrics_cuspajz).fillna(df.Lyrics_tekstovinet).isnull().sum()

0

In [46]:
lyrics_df = df.Lyrics_lyricstranslate.fillna(df.Lyrics_cuspajz).fillna(df.Lyrics_tekstovinet)

In [47]:
song_df = df.Song_lyricstranslate.fillna(df.Song_cuspajz).fillna(df.Song_tekstovinet)

In [48]:
lyrics = [l.splitlines() for l in lyrics_df]

In [49]:
all_lyrics = [", ".join(l) for l in lyrics if "https:" not in l]# and is_float(l[0])]

In [50]:
ref = re.compile(re.escape('ref '), re.IGNORECASE)
ref = re.compile(re.escape('ref. '), re.IGNORECASE)
refren = re.compile(re.escape('refren'), re.IGNORECASE)
pripjev = re.compile(re.escape('pripjev'), re.IGNORECASE)

In [51]:
corrected_lyrics=[]
for l in all_lyrics:
    l = ref.sub('', l)
    l = refren.sub('', l)
    l = pripjev.sub('', l)
    l = re.sub("\dx", "", l)
    l = re.sub("x\d", "", l)
    l = re.sub(" ,", "", l)
    l = re.sub(" ,", "", l)
    l = re.sub(" \.", "", l)
    l = re.sub(" ,", "", l)
    #l = re.sub("\.,", ". ", l)
    #l = re.sub(",.", ".", l)

    
    corrected_lyrics.append(l)


In [52]:
#classla.download('hr', type='nonstandard')
nlp = classla.Pipeline('hr', type='nonstandard') # run classla.download('hr', type='nonstandard') beforehand if necessary
#doc = nlp(".".join(corrected_lyrics))



2022-02-21 13:51:27 INFO: Loading these models for language: hr (Croatian):
| Processor | Package     |
---------------------------
| tokenize  | nonstandard |
| pos       | nonstandard |
| lemma     | nonstandard |
| depparse  | standard    |
| ner       | nonstandard |

2022-02-21 13:51:27 INFO: Use device: cpu
2022-02-21 13:51:27 INFO: Loading: tokenize
2022-02-21 13:51:27 INFO: Loading: pos
2022-02-21 13:51:32 INFO: Loading: lemma
2022-02-21 13:51:54 INFO: Loading: depparse
2022-02-21 13:51:58 INFO: Loading: ner
2022-02-21 13:51:58 INFO: Done loading processors!


In [53]:
df_nlp = pd.DataFrame()

for cl, song, song_id in zip(corrected_lyrics, song_df, df["Song_ID"].values):

    doc = nlp(cl)
    doc_dict = doc.to_dict()

    #df_song = pd.DataFrame([doc_dict[i][0] for i, _ in enumerate(doc_dict)]).transpose()
    sentences = [doc_dict[i][0] for i, _ in enumerate(doc_dict)]
    df_song = pd.DataFrame(sentences[0])
    for sentence in sentences[1:]:
        df_song = pd.concat([df_song, pd.DataFrame(sentence)])

    df_song["Song"] = song 
    df_song["Song_ID"] = song_id
    df_nlp = pd.concat([df_nlp, df_song])

In [54]:
df_nlp.reset_index(drop=True, inplace=True)

In [55]:
df_nlp

,id,text,lemma,upos,xpos,feats,head,deprel,ner,misc,Song,Song_ID
0,1,Nisam,biti,AUX,Var1s,Mood=Ind|Number=Sing|Person=1|Tense=Pres|VerbF...,2,cop,O,NaN,(R)evolucija,EOkBEcEU_1_0
1,2,spreman,spreman,ADJ,Agpmsnn,Case=Nom|Definite=Ind|Degree=Pos|Gender=Masc|N...,0,root,O,NaN,(R)evolucija,EOkBEcEU_1_0
2,3,vise,mnogo,ADV,Rgc,Degree=Cmp,2,advmod,O,NaN,(R)evolucija,EOkBEcEU_1_0
3,4,za,za,ADP,Sa,Case=Acc,5,case,O,NaN,(R)evolucija,EOkBEcEU_1_0
4,5,parole,parola,NOUN,Ncfpa,Case=Acc|Gender=Fem|Number=Plur,2,obl,O,SpaceAfter=No,(R)evolucija,EOkBEcEU_1_0
...,...,...,...,...,...,...,...,...,...,...,...,...
11748,23,nadam,nadati,VERB,Vmr1s,Mood=Ind|Number=Sing|Person=1|Tense=Pres|VerbF...,20,ccomp,O,SpaceAfter=No,Zivot nije bajka,EOkBEcEU_1_82
11749,24,",",",",PUNCT,Z,NaN,27,punct,O,NaN,Zivot nije bajka,EOkBEcEU_1_82
11750,25,sve,sve,PART,Qo,NaN,26,discourse,O,NaN,Zivot nije bajka,EOkBEcEU_1_82
11751,26,do,do,ADP,Sg,Case=Gen,27,case,O,NaN,Zivot nije bajka,EOkBEcEU_1_82


In [56]:
pos_tags = "ADJ – ADP – ADV – AUX – CCONJ – DET – INTJ – NOUN – NUM – PART – PRON – PROPN – PUNCT – SCONJ – SYM – VERB – X".split(" – ")
relations = "acl – advcl – advmod – advmod:emph – amod – appos – aux – case – cc – ccomp – compound – conj – cop – csubj – dep – det – det:numgov – discourse – dislocated – expl – fixed – flat – flat:foreign – iobj – list – mark – nmod – nsubj – nummod – obj – obl – orphan – parataxis – punct – root – vocative – xcomp".split(" – ")
word_features = "Animacy – Case – Definite – Degree – Foreign – Gender – Gender[psor] – Mood – Number – Number[psor] – NumType – Person – Polarity – Poss – PronType – Reflex – Tense – VerbForm – Voice".split(" – ")

In [57]:
features_df = pd.DataFrame()
rows = df_nlp.feats.str.split("|").to_list()

In [58]:
for i, row in enumerate(rows):
    if not type(row) == float:
        for f in row:
            features_df.loc[i, f[0:f.index("=")]] = f[f.index("=")+1:]


In [59]:
df_nlp = df_nlp.join(features_df)

In [60]:
df_nlp.to_csv(f"{artist_name}_nlp.csv")

In [61]:
df_nlp.columns

Index(['id', 'text', 'lemma', 'upos', 'xpos', 'feats', 'head', 'deprel', 'ner',
       'misc', 'Song', 'Song_ID', 'Mood', 'Number', 'Person', 'Tense',
       'VerbForm', 'Case', 'Definite', 'Degree', 'Gender', 'PronType',
       'Reflex', 'Polarity', 'Number[psor]', 'Poss', 'Animacy', 'Voice',
       'NumType', 'Gender[psor]', 'Foreign'],
      dtype='object')

In [62]:
df_nlp.Person.unique()

array(['1', nan, '3', '2'], dtype=object)

In [63]:
df_nlp[(df_nlp.Polarity=="Pos")]["lemma"].value_counts()[0:10]

da    2
Name: lemma, dtype: int64

In [64]:
df_nlp.ner.unique()

array(['O', 'B-PER', 'B-ORG', 'B-LOC', 'I-PER', 'B-DERIV-PER', 'B-MISC',
       'I-MISC', 'I-LOC', 'I-ORG'], dtype=object)

In [65]:
df_nlp[(df_nlp.ner=="B-LOC")]["lemma"].value_counts()[0:10]

Sava         4
Amerika      3
Hvar         3
sad          2
Triglav      2
prohladan    1
Čitluk       1
Neum         1
Višnjik      1
Vardar       1
Name: lemma, dtype: int64

In [66]:
df_nlp[(df_nlp.ner=="B-ORG")]["lemma"].value_counts()[0:10]

Stambol     3
živjeti     2
Facebook    1
Name: lemma, dtype: int64

In [67]:
df_nlp[(df_nlp.ner=="B-PER")]["lemma"].value_counts()[0:10]

KAJA           8
Bog            8
Pešta          6
Bečlija        6
Mladen         6
Beograđanka    4
kaja           4
Azra           3
Stambol        3
Krajina        2
Name: lemma, dtype: int64

In [68]:
df_nlp.upos.unique()

array(['AUX', 'ADJ', 'ADV', 'ADP', 'NOUN', 'PUNCT', 'CCONJ', 'SCONJ',
       'VERB', 'PRON', 'DET', 'PART', 'PROPN', 'INTJ', 'NUM', 'X'],
      dtype=object)

In [69]:
df_nlp[(df_nlp.upos=="SYM")]["lemma"].value_counts()[0:10]

Series([], Name: lemma, dtype: int64)

In [70]:
df_nlp[(df_nlp.upos=="PROPN")]["lemma"].value_counts()[0:10]

Ref            9
KAJA           8
Bog            8
Pešta          6
Bečlija        6
Mladen         6
Stambol        6
Azra           6
po             5
Beograđanka    4
Name: lemma, dtype: int64

In [71]:
df_nlp[df_nlp.lemma=="Zorz"].Song

Series([], Name: Song, dtype: object)

In [72]:
df_nlp[(df_nlp.upos=="VERB")]["lemma"].value_counts()[0:10]

znati      67
voljeti    59
moći       46
dati       41
doći       31
ljubiti    28
otići      27
nemati     25
čekati     23
vratiti    22
Name: lemma, dtype: int64

In [73]:
df_nlp[(df_nlp.upos=="NOUN")]["lemma"].value_counts()[0:10]

dan       51
ljubav    43
život     41
oko       39
srce      38
noć       33
drug      24
čovjek    20
tuga      20
kuća      18
Name: lemma, dtype: int64

In [88]:
df_nlp[df_nlp.lemma=="vojska"].Song.unique()

array([], dtype=object)

In [80]:
df_nlp[df_nlp.lemma=="tuga"].Song_ID.unique()

array(['EOkBEcEU_1_6', 'EOkBEcEU_1_8', 'EOkBEcEU_1_12', 'EOkBEcEU_1_19',
       'EOkBEcEU_1_31', 'EOkBEcEU_1_45', 'EOkBEcEU_1_48', 'EOkBEcEU_1_62',
       'EOkBEcEU_1_69', 'EOkBEcEU_1_78'], dtype=object)

In [86]:
df[df.Song_ID == "EOkBEcEU_1_6"].Lyrics_tekstovinet.values

array(["Ti si zadnja misao pred san\r\ni prva kad se probudim\r\nkako sporo prolazi mi dan\r\nkad nemam sta da izgubim\r\n\nLjubav putuje\r\nal' nekom drugom odlazi\r\nljubav putuje\r\ntu ne postoje razlozi\r\n \nRef.\r\nKad me pozdravis, ti i ne slutis\r\nod tvog osmijeha ja zivim danima\r\nali sta to sada vrijedi\r\ntaj biser u nizu\r\nto je samo sjaj u bijedi\r\nkad nisam ti blizu\r\n\nTi si zadnja misao pred san\r\ni prva kad se probudim\r\nkako sporo prolazi mi dan\r\nkad nemam sta da izgubim\r\n\nTuga putuje\r\npred mojim srcem zakoci\r\ntuga putuje\r\ntu razum nece pomoci\r\n\nRef."],
      dtype=object)

In [74]:
df_nlp.to_csv(f"{artist_name}_nlp.csv", index=False)

In [103]:
artist_name = "Magazin"
df_nlp_load = pd.read_csv(f"{artist_name}_nlp.csv")

In [101]:
artist_name = "Magazin"
df_load = pd.read_csv(f"data/{artist_name}_final.csv")

In [118]:
df_nlp_load[df_nlp_load.lemma=="Ana"].Song.unique()

array(['Kunigunda'], dtype=object)

In [105]:
df_nlp_load[df_nlp_load.lemma=="vojska"].Song_ID.unique()

AttributeError: 'DataFrame' object has no attribute 'Song_ID'

In [100]:
df_load[df_load.Song_ID == "Wm5tG4b5_1_19"].Lyrics_tekstovinet.values

array(['Od vremena jos od Krista\r\nnova lica, scena ista\r\nvrazje sile se trude\r\nda nas ne bude\r\n\nAntikristi i masoni\r\nkomunisti ovi, oni\r\nsire sotonske fraze\r\nda nas poraze\r\ne, moj narode, e, moj narode\r\n \nA mi gluhi, nijemi, slijepi\r\nsvatko u svom mraku strepi\r\nnarod biran od Boga\r\ngusi nesloga\r\n\nIz povijesti nase slavne\r\na i ove ne bas davne\r\nsve smo sto nam je vrijedno\r\nstekli zajedno\r\ne, moj narode, e, moj narode\r\n\nRef.\r\n    Uzmimo u ruke sudbinu i srecu\r\n   ni ja vise tako ne mogu ni necu\r\n    uzmimo u ruke sudbinu i srecu\r\n    e, moj narode, e, moj narode\r\n\nGeneracije junaka\r\ni pobjednicka vojska jaka\r\njos se brine i gine\r\nzbog domovine\r\n\nS nebeskih tih visina\r\ndaj nam Boze opet sina\r\nda nas vodi iz bijede\r\nsve do pobjede, e moj narode\r\n\n   Ref.'],
      dtype=object)

In [119]:
df_load[df_load.Song_tekstovinet == "Kunigunda"].Lyrics_tekstovinet.values

array(["Nije da necu, ma bas bi' htio\r\nime je, eto, nezgodno malo\r\nnemam za njega bas prikladne rime\r\nti odmah mislis da nije mi stalo\r\n\nDa si bar Ana, Marija, Tanja\r\nnesreca odmah bila bi manja\r\ncak da si neka Selma il' Kate\r\nopet bi' nesto lako smislio za te\r\n \nRef.\r\nKu, Ku, Kunigunda\r\nzbog imena mi zvoca i brunda\r\nKu, Ku, Kunigunda\r\nmuci me time da nadjem mu rime\r\n\nKu, Ku, Kunigunda\r\nma kad bi htjela da se strpi\r\nstvar vec hoda jer za par dana\r\nstizu rime iz Japana\r\n\nPostaje teze zivjeti s njom\r\ngrozi se, prijeti, ucinit' ce lom\r\ndig'o bi' pozdrav peru papiru\r\nma kad bi htjela barem pustit' me na miru\r\n\nDa sam bar Ana, Marija, Tanja\r\nnesreca odmah bila bi manja\r\ncak da sam neka Selma il' Kate\r\nopet bi' nesto lako smislio za te\r\n\nRef. 2x"],
      dtype=object)